# ICS2203 – Statistical Natural Language Processing
## Building a Language Model

### Necessary Imports

In [1]:
import xml.etree.ElementTree as ET
import os
import random

### Part 1

#### Extracting data from the corpus.

Reading raw data from the Download files provided.

In [2]:
file_directory = './Data/Texts/'
all_data = []

#appends all files to all_data
for x in os.listdir(file_directory):
    for y in os.listdir(file_directory + x):
        all_data.append(ET.parse(file_directory+x+'/'+y))


Iterating through every xml file in the dataset and extracts all text from the files. <br>
The data is saved in a 2D list. Every sub-list in the main list will contain 1 sentence. Every element in the sublist will contain 1 word or punctuation mark. <br>
Some words ended with a space in the corpus, so I made sure to negate that space so that, for example: 'hello' and 'hello ' count as the same word. <br>
I added start- and end-of-sentence tags to ease computation later on. <br> The dataset was then shuffled and a random 15% of the dataset was taken to be the testing set.

In [3]:
def extract(data):
    dataset = []
    for file in data:
        for tree in file.getroot():
            for sentence in tree.iter('s'):
                temp_sentence = ['<s>']
                for word in sentence:
                        if word.text is not None:
                            if word.text[-1] == ' ':
                                temp_sentence.append(word.text[:-1])
                            else:
                                temp_sentence.append(word.text)
                temp_sentence.append("</s>")
                dataset.append(temp_sentence)

    random.shuffle(dataset)
    slicing_point = int(len(dataset) * 0.85)
    return dataset[:slicing_point], dataset[slicing_point:] 

#### Calculating N-Grams

The following function takes a dataset, the n (amount of elements in the n-gram), and if, provided, it takes a dictionary with already defined words and frequencies. <br>
This method iterates through every sentence in the training dataset and generates n-grams for a provided n. <br>
These n-grams are then tallied into a dictionary, where every n-gram is saved as a key, and the amount of times it appears is its value. <br>
This dictionary is then returned.

In [5]:
def calculate_n_gram(data, n, frequencies = {}):
    for sentence in data:
        #len(sentence)-n+1 was taken as for each n-gram, there are that many windows of words(grams)
        temp_ngrams = [tuple(sentence[x:x+n]) for x in range(len(sentence)-n+1)]
        for ngram in temp_ngrams:
            if ngram in frequencies:
                frequencies[ngram] += 1
            else:
                frequencies[ngram] = 1
    
    return frequencies


#### Testing Methods

This method sorts the dictionary items in order of values first, but showing both values and keys. This was used mainly to check my work and test, but I saw it fit to keep it here, as it was something I used.

In [6]:
def sort_dict_showing_items(unsorteddict):
    return dict(sorted(unsorteddict.items(), key=lambda x: x[1], reverse=True))

### Part 2

#### Markov???

#### Vanilla Model

In [7]:
def calculate_vanilla(dataset, n):
    vanilla = {}
    n_grams = calculate_n_gram(dataset, n)
    total_grams = sum(n_grams.values())
    for key, value in n_grams.items():
        vanilla.update({key: value/total_grams})
    return vanilla

#### Laplace Model

The smooth method creats every n-gram that could be possible for a value n where n < 3. I coded different options for n == 1, n == 2 and n == 3, as opposed to recusrively creating for loops to make it work for any value of n since my program will only be used for n-grams up to trigrams. <br>
For n == 1, all possible words in the corpus can are permitted. <br>
For n == 2, the first word cannot be an end-of-sentence tag and the second word cannot be a start-of-sentence tag. <br>
For n == 3, the first word of the trigram can't be an end-of-sentence tag, the second word of the trigram can't be a start- or end-of-sentence tag and the third word of the trigram can't be a start-of-sentence tag.

In [8]:
def smooth(dataset, n):
    #creating my vocabulary
    vocabulary = []
    for sentence in dataset:
           for word in sentence:
                if word not in vocabulary:
                    vocabulary.append(word)

    all__possible_ngrams = {}

    #if unigram
    if n == 1:
      
      all__possible_ngrams = {x: 1 for x in vocabulary}

    if n==2:
        for x in vocabulary:
            for y in vocabulary:
                all__possible_ngrams.update({(x,y): 1})             

        #Taking out multiple sentence start and end tags in a row, since this would make no sense.
        all__possible_ngrams.update({('<s>', '<s>'): 0})
        all__possible_ngrams.update({('</s>', '</s>'): 0})

    if n == 3:
        for x in vocabulary:
            #1st element of the trigram can't be an end of sentence tag
            if x == '</s>':
                continue
            for y in vocabulary:
                # 2nd element of the trigram can't be a start or end of sentence tag
                if y == '<s>' or y == '</s>':
                    continue
                for z in vocabulary:
                    # 3nd element of the trigram can't be a start of sentence tag
                    if z == '<s>':
                        continue
                    
                    all__possible_ngrams.update({(x, y, z): 1})

        # Taking out multiple sentence start and end tags in a row, since this would make no sense.
    return all__possible_ngrams
    

The 'calculate_laplace' function takes the training set and n as parameters. <br> It first passes these parameters to the smooth method to generate the laplace smoothing effect. It then passes the resultant dictionary as well as the function parameter to the 'calculate_n_grams' function to generate frequencies for all of the corupus' n-grams. <br>
The frequencies were then divided by the total n-grams in the corpus to tranform the frequencies into probabilities. 

In [9]:
def calculate_laplace(dataset, n):
    laplace = smooth(dataset, n)
    laplace = calculate_n_gram(dataset, n, laplace)
    total_grams = sum(laplace.values())
    for key, value in laplace.items():
        laplace.update({key: value/total_grams})
    return laplace


#### UNK and Laplace Model

To implement the UNK model, I iterated through every n-gram and for each one, if the value is less than or equal to 2, I added it to a counter of UNKs. If there are more than 2 occurences of that n-gram, Laplace smoothing is worked out and the result is added to the final dictionary. Finally, Laplace smoothing is calculated on the UNK counter and it is also added to the dictionary.

In [10]:
def calculate_unk(dataset, n):
    unk = {}
    unks = 0
    n_grams = calculate_n_gram(dataset, n)
    total_grams = sum(n_grams.values())
    vocabulary = len(n_grams)
    for key, value in n_grams.items():
        if value <= 2:
            unks += value
        else:
            unk.update({key: value+1/total_grams+vocabulary})
            
        unk.update({'<UNK>': unks+1/total_grams+vocabulary})
    return unk


#### Linear Interpolation

Linear Interpolation is calculated by going through a given sentence and calaculating the probabilities for every unigram, bigram and trigram in the sentence for a given flavour of language model (Vanilla, Laplace or UNK). The user chooses which flavour and sentence they would like to use and these, along with the previously-computed dictionary of probabilities are passed as a parameter.

In [15]:
def linear_interpolation(input_sentence, language_models_data, flavour):
    
    trigram_lambda = 0.6
    bigram_lambda = 0.3
    unigram_lambda = 0.1    

    #Cleaning the input (making sure LM flavour names were in the same format they are in in the dictionary)
    if flavour == 'Vanilla' or 'VANILLA':
        flavour = 'vanilla'

    if flavour == 'Laplace' or 'LAPLACE':
        flavour = 'laplace'

    if flavour == 'unk':
        flavour = 'UNK'

    probability = 1

    tokenised_sentence = ("<s> " +input_sentence+ " </s>").split()

    #len(tokenised_sentence) - 2 was chosen as in a sentence of len x, there are x - 2 trigrams
    for i in range(len(tokenised_sentence)-2):
        
        #accessing the previously-computed probabilities stored in the dictionary
        prob_trigram = language_models_data[flavour][(tokenised_sentence[i], tokenised_sentence[i+1], tokenised_sentence[i+2])]
        prob_bigram = language_models_data[flavour][(tokenised_sentence[i+1], tokenised_sentence[i+2])]
        prob_unigram = language_models_data[flavour][(tokenised_sentence[i+2])]

        #multiplying the probability by the given lambda
        probability *= unigram_lambda*prob_unigram + bigram_lambda*prob_bigram + trigram_lambda*prob_trigram
        
    return probability


#### Running the Functions Above

In [ ]:
training_set, test_set = extract(all_data)

In [ ]:
all_language_models = {'vanilla': {
                                    'unigram': calculate_vanilla(training_set, 1),
                                    'bigram': calculate_vanilla(training_set, 2),
                                    'trigram': calculate_vanilla(training_set, 3)
                        },

                        'laplace': {
                                    'unigram': calculate_laplace(training_set, 1),
                                    'bigram': calculate_laplace(training_set, 2),
                                    'trigram': calculate_laplace(training_set, 3)
                        },

                        'UNK': {
                                'unigram': calculate_unk(training_set, 1),
                                'bigram': calculate_unk(training_set, 2),
                                'trigram': calculate_unk(training_set, 3)
                        }
}

#### Evaluation

In [ ]:
def calculate_perplexity():
    pass

def calculate_probability(sentence, n, language_dict, flavour):
    # Cleaning the input (making sure LM flavour names were in the same format they are in in the dictionary)
    if flavour == 'Vanilla' or 'VANILLA':
        flavour = 'vanilla'

    if flavour == 'Laplace' or 'LAPLACE':
        flavour = 'laplace'

    if flavour == 'unk':
        flavour = 'UNK'

    if n == 1:
        gram = 'unigram'

    if n == 2:
        gram = 'bigram'
    
    if n == 3:
        gram = 'trigram'

    probability = 1
    tokenised_sentence = ("<s> " + sentence + " </s>").split()

    temp_ngrams = [tuple(tokenised_sentence[x:x+n]) for x in range(len(tokenised_sentence)-n+1)]

    for x in temp_ngrams:
        probability *= language_dict[flavour][gram][x]

    return probability